# Table of Contents

1. Introduction [purpose, agenda, alignment]
    - What are GANs? 
    - the outline 
    - What do you know about GANs?
2. Fundamental Concept [key definitions, concepts]
    - GAN breakdown
    - Components
3. Deep Dive [architecture and workflow, step on how it works]
    - whats the architecture
    - How it works
    - How to build one
4. Real-world applications [use cases, how they impact your business/product]
    - use cases
    - impact on the business
5. Interactions [Questions&Answers, polls, scenario discussion]
    - QnA
    - polls
    - scenario discussion
6. Hands-On [case study, practical implementation]
    - case study
    - implementation
7. Conclusion [summary, resources, what to do next]
    - summary pointers
    - resources
    - what next

# 1. Introduction [purpose, agenda, alignment]

## What are GANs?

<img src='g1.png' />

In [ ]:
# What do you know about GANs?

no

simple example of GANs

this person done not exists: https://thispersondoesnotexist.com/

In [ ]:
a neural network architecture that will generate new data

In [ ]:
Coke ad generated by AI: https://www.facebook.com/watch/?v=1276572062941149

In [ ]:
Inventor of GANs: Ian J. Goodfellow

https://scholar.google.ca/citations?user=iYN86KEAAAAJ&hl=en

# 2. Fundamental Concept [key definitions, concepts]

### GAN breakdown and Components

<img src='g2.png' />

In [ ]:
2 neural networks are in GANs

1 - Generator: it will generate realistically fake values to fool the discriminator
    - the network takes the random noise as input and produce data
    - the goal is to generate data that is as close as possible to real data
    
2 - Discriminator: it will differentiate betweek fake and real values
    - the network takes the real data and the data generated by Generator as input
    - it differentiate between the two
    - it outputs the probability that the given data is real

# 3. Deep Dive [architecture and workflow, step on how it works]

### whats the architecture

<img src='g4.png' />


### How it works

<img src='g11.png' />

In [ ]:
Generator

input - random noise vector (sampled normal distribution) - starting point of data generation

fully connected layers - used to transform the input noise vector into a suitable shape 

batch normalization - this technique is used between layers to stabilize learning 
                        by normalizing the ouput of previous layer

activation function - ReLU activation

transposed convolutional layer(deconvolutional layer) - 
            they upsample the input from the previous layer to a high dimension
    the opposite of convolutional layer
    
reshaping layer - reshape it in desired output format

output layer - having tanh activation function 
            - tanh is used to output pixel values in a normalized range [-1,1]




In [ ]:
Discriminator

input - data samples which is either real or fake

convolutional layer - helps in extracting features from the input image

batch normalization - to stabilize learning

activation function - Leaky ReLU activation
                - it allows a small gradient when the unit is not active
                - to maintain the gradient flow during training
    
pooling layer - reduce the dimensions of the input data

fully connected layer - used to process the features extracted by convolutional layer

output - scalar value between 0 and 1 (probability of the input sample as real)

### How to build one

Basic implementation in pytorch on MNIST dataset

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.utils import save_image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_size = 64
lr = 0.0002
z_dim = 100
num_epochs = 20
image_size = 28 * 28

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])

mnist = datasets.MNIST(root="./data", train=True, transform=transform, download=True)
dataloader = DataLoader(dataset=mnist, batch_size=batch_size, shuffle=True)

class Generator(nn.Module):
    def __init__(self, input_dim):
        super(Generator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, image_size),
            nn.Tanh()
        )

    def forward(self, z):
        return self.fc(z).view(-1, 1, 28, 28)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(image_size, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        return self.fc(img_flat)

generator = Generator(z_dim).to(device)
discriminator = Discriminator().to(device)

criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=lr)
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr)

def generate_noise(batch_size, z_dim):
    return torch.randn(batch_size, z_dim).to(device)

for epoch in range(num_epochs):
    for batch_idx, (real_images, _) in enumerate(dataloader):
        real_images = real_images.to(device)
        batch_size = real_images.size(0)
        
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)
        
        optimizer_D.zero_grad()
        real_output = discriminator(real_images)
        real_loss = criterion(real_output, real_labels)
        
        noise = generate_noise(batch_size, z_dim)
        fake_images = generator(noise)
        fake_output = discriminator(fake_images)
        fake_loss = criterion(fake_output, fake_labels)
        
        d_loss = real_loss + fake_loss
        d_loss.backward()
        optimizer_D.step()

        optimizer_G.zero_grad()
        noise = generate_noise(batch_size, z_dim)
        fake_images = generator(noise)
        fake_output = discriminator(fake_images)
        
        g_loss = criterion(fake_output, real_labels)
        g_loss.backward()
        optimizer_G.step()

        if batch_idx % 100 == 0:
            print(f"Epoch [{epoch}/{num_epochs}] Batch {batch_idx}/{len(dataloader)} \
                  Loss D: {d_loss.item()}, Loss G: {g_loss.item()}")
    
    with torch.no_grad():
        noise = generate_noise(64, z_dim)
        fake_images = generator(noise)
        save_image(fake_images, f"gan_images_epoch_{epoch}.png", normalize=True)

with torch.no_grad():
    noise = generate_noise(64, z_dim)
    fake_images = generator(noise)
    save_image(fake_images, "final_gan_images.png", normalize=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:03<00:00, 2881119.21it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 134687.40it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



...

Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4344476.34it/s]


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw

Epoch [0/20] Batch 0/938                   Loss D: 1.3752137422561646, Loss G: 0.7055636644363403
Epoch [0/20] Batch 100/938                   Loss D: 0.31725192070007324, Loss G: 1.967050552368164
Epoch [0/20] Batch 200/938                   Loss D: 0.1537465751171112, Loss G: 3.3735804557800293
Epoch [0/20] Batch 300/938                   Loss D: 0.6559709310531616, Loss G: 10.448826789855957
Epoch [0/20] Batch 400/938                   Loss D: 0.00040623178938403726, Loss G: 11.698454856872559
Epoch [0/20] Batch 500/938                   Loss D: 0.05595643073320389, Loss G: 9.931100845336914
Epoch [0/20] Batch 600/938                   Loss D: 0.013205916620790958, Loss G: 7.84675931930542
Epoch [0/20] Batch 700/938                   Loss D: 0.0035287761129438877, Loss G: 6.478899002075195
Epoch [0/20] Batch 800/938                   Loss D: 0.012762698344886303, Loss G: 8.530923843383789
Epoch [0/20] Batch 9

Epoch [8/20] Batch 200/938                   Loss D: 0.2576775550842285, Loss G: 5.129783630371094
Epoch [8/20] Batch 300/938                   Loss D: 0.23205086588859558, Loss G: 4.131558418273926
Epoch [8/20] Batch 400/938                   Loss D: 0.23609159886837006, Loss G: 4.478598594665527
Epoch [8/20] Batch 500/938                   Loss D: 0.19304409623146057, Loss G: 3.796693801879883
Epoch [8/20] Batch 600/938                   Loss D: 0.4978059232234955, Loss G: 5.269026279449463
Epoch [8/20] Batch 700/938                   Loss D: 0.06880423426628113, Loss G: 5.254241466522217
Epoch [8/20] Batch 800/938                   Loss D: 0.0813046544790268, Loss G: 5.640722274780273
Epoch [8/20] Batch 900/938                   Loss D: 0.2547832727432251, Loss G: 4.055717945098877
Epoch [9/20] Batch 0/938                   Loss D: 0.1305491328239441, Loss G: 4.923584461212158
Epoch [9/20] Batch 100/938                   Loss D: 0.0895596519112587, Loss G: 5.67410945892334
Epoch [9/

Epoch [16/20] Batch 400/938                   Loss D: 0.4964970350265503, Loss G: 3.30446720123291
Epoch [16/20] Batch 500/938                   Loss D: 0.4248071014881134, Loss G: 3.4845221042633057
Epoch [16/20] Batch 600/938                   Loss D: 0.16023211181163788, Loss G: 4.49136209487915
Epoch [16/20] Batch 700/938                   Loss D: 0.44253212213516235, Loss G: 2.7773475646972656
Epoch [16/20] Batch 800/938                   Loss D: 0.27827656269073486, Loss G: 4.026714324951172
Epoch [16/20] Batch 900/938                   Loss D: 0.7150179147720337, Loss G: 5.87007474899292
Epoch [17/20] Batch 0/938                   Loss D: 0.08823823928833008, Loss G: 3.6295151710510254
Epoch [17/20] Batch 100/938                   Loss D: 0.42300981283187866, Loss G: 3.2442479133605957
Epoch [17/20] Batch 200/938                   Loss D: 0.24984610080718994, Loss G: 3.173337459564209
Epoch [17/20] Batch 300/938                   Loss D: 0.6430570483207703, Loss G: 2.48758029937

# 4. Real-world applications [use cases, how they impact your business/product]

## use cases

### 1. Pose guided person image generation

<img src='g5.png' />
<img src='g6.png' />
<img src='g10.png' />

### 2. StarGAN

<img src='g7.png' />

<img src='g8.png' />

### 3. Super resolution

<img src='g9.png' />
<img src='g3.png' />



### impact on the business

10 GANs use cases in 2024: https://research.aimultiple.com/gan-use-cases/

# 5. Interactions [Questions&Answers, polls, scenario discussion]

### QnA or polls

TGAN - Tabular GAN -> https://github.com/sdv-dev/TGAN

In [ ]:
Are GANs used in tabular data?

- TGAN - use case - mimic the distribution of real tabular data

advantages
- data generation when you have limited dataset
- you have imbalanced data - you can generalize it using GANs
- anomaly detection - generate normal samples and indentify deviations


mode collapse 
- if the data contains less categories or classes it will be difficult to generate tabular data


### business scenario discussion

#### Transforming image-based retail business with GANs

Overview: 

Imagine you are part of the leadership team at a major online retail company that specializes in fashion. 

The company has been exploring AI-based solutions to enhance customer experience, particularly around virtual try-ons and personalized recommendations. 

You're already leveraging deep learning models for product recommendations and some basic image processing tasks. 

<img src='g12.gif' />

In [ ]:
do you feel the need of implementing GANs in this particular use case?

engineers question - is there any traditional way of doing the same?



In [ ]:
if you want to make any AI implementation successfull

- check if the solution is irreversible
- 

In [ ]:
what concerns might rise from using GANs in fashion?

how can the data used to train GANs may introduce bias and 
    how it will affect the customer satisfaction or the brand perception
- monitoring

In [ ]:
how you are going to justify the computational cost investment and ROI?

compair traditional and new approaches to comeup with right decision

engineers q: what infrastrucutre, skills, and tools are needed to make is a success
    
    

# 6. Hands-On [case study, practical implementation]


### case study

### implementation

List of applications and demos: https://github.com/nashory/gans-awesome-applications



### In production - AWS

In [ ]:
- Amazon SageMaker
- Amazon Bedrock

# Conclusion [summary, resources, what to do next]


### summary pointers


### resources

Research papers
- https://arxiv.org/abs/1406.2661

Blogs
- https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
- https://www.kaggle.com/code/songseungwon/pytorch-gan-basic-tutorial-for-beginner

Video
- https://www.youtube.com/watch?v=xBX2VlDgd4I

### what next

You can work on these projects to add to your portfolio

- Image restoration: https://github.com/TencentARC/GFPGAN
- Pose with Style: https://github.com/BadourAlBahar/pose-with-style
- Generating Anime character(Colab Notebook): https://colab.research.google.com/drive/15EGSIv_jZxDvYWODNwEG-I4pV9i3RwlN?usp=sharing
- Text to Image generation by NVIDIA: https://blogs.nvidia.com/blog/gaugan2-ai-art-demo/
